# Imports

In [ ]:
import requests
from pprint import pprint

# Initialize
Note: 
1. `API_KEY` should be the same as the one that was given docker-compose env variable
2. `baseurl` refers the internal name of the docker container with docker-compose's network. This has be changed to `localhost` if the notebook is run outside the jupyter container env.

In [ ]:
baseurl = "http://image_service:8000"
API_KEY = "foobar"
headers = {"x-api-key" : API_KEY}

# Generate Upload Link
Takes expiry interval and generates a upload_id that can used to upload the images to. `expiry` - expiry of the requested upload link (in seconds) 

In [ ]:
expiry = 120
payload = {
    "expiry" : expiry
}
url = "/api/generate-upload-link"
response = requests.post(baseurl + url, json=payload, headers=headers)
print("Response - ", response.status_code)
upload_id = None
if response.ok:
    response = response.json()
    upload_id = response["upload_id"]
print("Upload ID - ", upload_id)

# Upload Images
Uploads the images mentioned in the `image_files` variable (as list path to the image files) to the `upload_id` generated from the above step. There are sample images in the `sample_images` folder which is accessible from within this jupyter container. If more images needs to be added, they can be added to `sample_images` folder in the local repo folder and they should be available to this container.

In [ ]:
image_files = [
    "sample_images/58729zbpf682b933xyjhzhge7c.jpg", 
    "sample_images/PXL_20240505_213734915.jpg", 
    "sample_images/PXL_20240506_093352908.jpg"
]
upload_files = [("image_files", open(f, "rb")) for f in image_files]

url = f"/api/upload-images/{upload_id}"
print("Calling - ", baseurl + url)
response = requests.post(baseurl + url, files=upload_files, headers=headers)
for _,f in upload_files:
    f.close()
print("Response - ", response.status_code)
response_data = response.json()
print("Response Json", response_data)
images = {}
if response.ok:
    images = response_data
    for k, v in images.items():
        print(k, " - ", v)

# Images API
This is a static api. The code blocks fetches the image and renders them if they are there. Else a broken image will be shown if the url is invalid. 
This does not require authentionca headers. 
Note: This baseurl is `localhost` in this notebook , as this `Image` and `display` function directly calls from the browser.

In [ ]:
from IPython.display import Image, display
url = "http://localhost:8001/api/images/"
for image_name, image_id in images.items():
    print("Displaying image_name - ", image_name, " image_id - ", image_id)
    display(Image(url=url + image_id, width=200, unconfined=True))

# Stats
Stats is a straightforward api with no parameters. It fetches pre set list of stats namely `Image Upload Frequency Per Day (last 30 days)` , `Most Popular Image Format` and `Top 10 Popular Camera Models`

In [ ]:
url = "/api/stats"
response = requests.get(baseurl + url, headers=headers)
if response.ok:
    stats = response.json()
    pprint(stats)